In [12]:
from pylate import indexes, models, retrieve
from setretrieval.utils.utils import pickload
from statistics import mean
from matplotlib import pyplot as plt
from datasets import Dataset, DatasetDict, load_dataset
from tqdm import tqdm
from setretrieval.utils.utils import pickdump
from sentence_transformers import SentenceTransformer
from transformers import AutoModelForCausalLM, AutoTokenizer


In [2]:
dataset = Dataset.load_from_disk("../data/datastores/wikipedia_docs_15k")
# try to take another node...
question_set = DatasetDict.load_from_disk("../data/colbert_training/gemini_ntrain_ptest")['train'].select(range(5200, 10000))
res = pickload("../cache/gendata/passagesearchtrain_v1_5.2k_0.pkl")

In [3]:
def chunks_to_inds(clist):
    # given list of booleans, return indices where True
    return [i for i, c in enumerate(clist) if c]


In [14]:
allpos = []
for r in tqdm(res, desc="Converting to indices"):
    # each r is a list of 
    inds = chunks_to_inds(r)
    allpos.append([dataset[ind]['text'] for ind in inds])

Converting to indices: 100%|██████████| 2347/2347 [01:07<00:00, 35.01it/s]


In [ ]:
pickdump(allpos, "../cache/gendata/passagesearchtrain_v1_5.2k_4Bpos.pkl")

In [26]:
alltext = [dataset[ind]['text'] for ind in range(len(dataset))]

In [ ]:
print(len(res))

2347


In [ ]:
[sum(r) for r in res].count(0)

2

In [ ]:
model = models.ColBERT(
    model_name_or_path="../cache/colbert_training/contrastive-Qwen_Qwen¬3-Embedding-0.6B-bs8-e11-lr3e-05-gemini_ntrain_ptest/checkpoint-500",
)

In [ ]:
index = indexes.PLAID(
    index_folder="../cache/colbert_indices",
    index_name="indextest",
    override=False
)

retriever = retrieve.ColBERT(index=index)

In [ ]:
documents_ids = ["1", "2", "3"]

documents = [
    "ColBERT’s late-interaction keeps token-level embeddings to deliver cross-encoder-quality ranking at near-bi-encoder speed, enabling fine-grained relevance, robustness across domains, and hardware-friendly scalable search.",

    "PLAID compresses ColBERT token vectors via product quantization to shrink storage by 10×, uses two-stage centroid scoring for sub-200 ms latency, and plugs directly into existing ColBERT pipelines.",

    "PyLate is a library built on top of Sentence Transformers, designed to simplify and optimize fine-tuning, inference, and retrieval with state-of-the-art ColBERT models. It enables easy fine-tuning on both single and multiple GPUs, providing flexibility for various hardware setups. PyLate also streamlines document retrieval and allows you to load a wide range of models, enabling you to construct ColBERT models from most pre-trained language models.",
]

# Encode the documents
documents_embeddings = model.encode(
    documents,
    batch_size=1,
    is_query=False, # Encoding documents
    show_progress_bar=True,
)

# Add the documents ids and embeddings to the PLAID index
index.add_documents(
    documents_ids=documents_ids,
    documents_embeddings=documents_embeddings,
)

In [ ]:
queries_embeddings = model.encode(
    ["query for document 3", "query for document 1"],
    batch_size=32,
    is_query=True, # Encoding queries
    show_progress_bar=True,
)
scores = retriever.retrieve(
    queries_embeddings=queries_embeddings,
    k=10
)
print(scores)